In [3]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


# SETUP ENVIRONMENT

## MNT DRIVE AND CLONE GITHUB REPO

In [4]:
# mnt drive 
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# Access token from drive without exposing in text (accsessible in logfiles lol. Better than nothing)
import os
import subprocess
from pathlib import Path

drive_path = Path('/content/drive/MyDrive/bioNLP/')
token = open(Path(drive_path.parent / "token/colab.txt"), "r").read()
tkn = token.split("=")

repo_name = "callebalik/clinical_NLP_SE.git"
cmd_string = 'git clone https://{0}@github.com/{1}'.format(tkn[1], repo_name)

!{cmd_string}
cmd_string = "" # removing the variable
print(cmd_string)

Cloning into 'clinical_NLP_SE'...
remote: Enumerating objects: 490, done.
remote: Counting objects: 100% (490/490), done.
remote: Compressing objects: 100% (332/332), done.
remote: Total 490 (delta 210), reused 356 (delta 110), pack-reused 0
Receiving objects: 100% (490/490), 3.38 MiB | 12.25 MiB/s, done.
Resolving deltas: 100% (210/210), done.



## Set paths for the environment and import models

In [6]:
# Set paths for the colab environment 
import os 
import sys
from pathlib import Path

repo_path = Path('/content/clinical_NLP_SE/')

models_path = Path(drive_path / 'models')
data_path = repo_path / 'data/'

sys.path.append(str(repo_path / 'sripts/')) # Let pyhton find scripts from the repo

In [7]:
# Make colab update/reaload modules when cells run
%load_ext autoreload
%autoreload 2

## Install requirements 

In [8]:
%%shell 
#Installed from requirements, but left as a note
# pip install spacy-transformers
# updating spacy in colab 
# pip install spacy==3.0.6  

pip install -r /content/clinical_NLP_SE/requirements.txt
python -m spacy info

     |████████████████████████████████| 51kB 4.1MB/s 
     |████████████████████████████████| 12.8MB 228kB/s 
     |████████████████████████████████| 13.5MB 123kB/s 
     |████████████████████████████████| 71kB 11.6MB/s 
     |████████████████████████████████| 460kB 44.5MB/s 
     |████████████████████████████████| 1.0MB 39.9MB/s 
     |████████████████████████████████| 2.1MB 46.5MB/s 
     |████████████████████████████████| 1.1MB 34.1MB/s 
     |████████████████████████████████| 9.1MB 38.9MB/s 
     |████████████████████████████████| 51kB 7.9MB/s 
     |████████████████████████████████| 194kB 59.4MB/s 
     |████████████████████████████████| 901kB 40.1MB/s 
     |████████████████████████████████| 3.3MB 42.5MB/s 
     |████████████████████████████████| 122kB 56.7MB/s 
  Created wheel for smart-open: filename=smart_open-3.0.0-cp37-none-any.whl size=107098 sha256=00ed9afe9a99bf74dd1a125536bf7b8f557f958dc39885ad548c4d4c2944ff24
  Stored in directory: /root/.cache/pip/wheels/18/88/7c/f06da

## GET DATA AND MODELS


In [9]:
%%shell
# Get models from Kungbib https://github.com/Kungbib/swedish-spacy

#Transformer models - # N.B. Make sure to install the spacy-transformers extension, or the models won't work.
#wget https://data.kb.se/datasets/2020/10/swedish_nlp/spacy/sv_tagger-0.0.0.tar.gz

pip install "/content/drive/MyDrive/bioNLP/models/sv_pipeline-0.0.0.tar.gz"

# Older models
#wget https://data.kb.se/datasets/2020/10/swedish_nlp/spacy/sv_model_upos.zip
#wget https://data.kb.se/datasets/2020/10/swedish_nlp/spacy/sv_model_xpos.zip


Processing ./drive/MyDrive/bioNLP/models/sv_pipeline-0.0.0.tar.gz
  Created wheel for sv-pipeline: filename=sv_pipeline-0.0.0-cp37-none-any.whl size=935375398 sha256=b32cda88e7af7e65ebf88c574e3d6ca9b40ff262b26f5eb36059377adea5d33b
  Stored in directory: /root/.cache/pip/wheels/78/6b/1a/3724784df18ac433f7975f29d5cb3c47c3cfa3f40c4a87f60d
Successfully built sv-pipeline


### Getting the Swedish models https://data.kb.se/datasets/2020/10/swedish_nlp/spacy/

https://github.com/Kungbib/swedish-spacy

The models can be downloaded with the commands below. For colab this is slow and they are instead accessed from the mounted from a google drive where they have been downloaded. If first time download and add to the correct folder of your drive. 


### Note from Kungbib

** UPDATE February 2021 **: We are adding two transformer-based models trained in spaCy 3.0. They are available to download at the same link given below.
One model is a complete pipeline with UPOS tagger, parser, sentencer, ner and lemmatizer (sv_pipeline-0.0.0.tar.gz). Unfortunately the lemmatizer is not yet trainable in spaCy, so the performance is as good as the quality of the rules/lookup tables available for Swedish (i.e. not very good). 

If you need a Swedish lemmatizer we advise you for the moment to have a look at Stanza, efselab or lemmy.ww

---


The other model is a XPOS tagger in case you need language-specific part-of-speech tags (sv_tagger-0.0.0.tar.gz).

## INITIALIZE SPACY

In [10]:
import spacy 
nlp = spacy.load("sv_pipeline") # Load the swedish model (can be done this way since it's installed as a pip package)

# MODEL (Colab: Ctrl + F8 in this cell to run all setup above)
*Time estimate 3 min*

## ICD-10

In [11]:
''' Processing ICD codes using regex 

pattern and special signs 
LDD   LLLLLL
LDD † LLLLLL
LDD * LLLLLL

Example 
P75	*	Mekoniumileus vid cystisk fibros (E84.1)


'''

'''
Positive Lookbehind (?<=\D\d\d) Assert that the Regex below matches
  \D matches any character that's not a digit (equivalent to [^0-9])
  \d matches a digit (equivalent to [0-9])
  \d matches a digit (equivalent to [0-9])
\s matches any whitespace character (equivalent to [\r\n\t\f\v ])
+? matches the previous token between one and unlimited times, as few times as possible, expanding as needed (lazy)
. matches any character (except for line terminators)
? matches the previous token between zero and one times, as many times as possible, giving back as needed (greedy)
\s matches any whitespace character (equivalent to [\r\n\t\f\v ])

Global pattern flags
g modifier: global. All matches (don't return after first match)
m modifier: multi line. Causes ^ and $ to match the begin/end of each line (not only begin/end of string)
'''

import re

regex_ddd = '/(?<=\D\d\d)\s+?.?\s/gm'
regex_dddR = '/(?<=R\d\d)\s+?.?\s/gm' 
ddd = r'(?<=\D\d\d)\s+?.?\s'
dddd = r'(?<=\D\d\d\d)\s+?.?\s'
ddddd = r'((?<=\D\d\d\d\d)|(?<=\D\d\d\d\D))\s+?.?\s'

'''
A513B	†	Syfilitisk (sekundär) alopeci (L99.8)
Y3498		Ospecificerad skadehändelse, med oklar avsikt-plats, ospecificerad-andra specificerade aktiviteter
Y3499		Ospecificerad skadehändelse, med oklar avsikt-plats, ospecificerad-aktivitet, ospecificerad
Y586A		Komplikation av vaccin mot kikhosta, enbart (P)
Y586B		Komplikation av vaccin mot difteri, kikhosta och stelkramp, kombinerat (DPT)
Y586W		Komplikation av annat kombinerat 
'''

ICD_PATH = data_path / 'raw/codes/icd-10-se-2021-text'
file_path = ICD_PATH / 'digit3.txt'

ICD000 = []
with open(ICD_PATH / 'digit3.txt','r') as codes:
    for line in codes:
        x = re.split(ddd, line)
        ICD000.append(x[1].strip())

ICD0000 = []
with open(ICD_PATH / 'digit4.txt','r') as codes:
    for line in codes:
        x = re.split(dddd, line)
        ICD0000.append(x[1].strip())
        
ICD00000 = []
with open(ICD_PATH / 'digit5.txt','r') as codes:
    for line in codes:
        x = re.split(ddddd, line)
        ICD00000.append(x[1])

ICD = ICD000 + ICD0000 + ICD00000




## Negations


In [12]:
''' Processing negations '''

NEG_PATH = data_path / 'raw/negations'

NEG = []
with open(NEG_PATH / 'negEx2.txt','r') as neg:
    for line in neg:
        x = re.split(r'\s+(?=\[)', line)
        NEG.append(x[0].strip())

## Add matcher from dictionaries 

To create the patterns, each phrase has to be processed with the nlp object. If you have a trained pipeline loaded, doing this in a loop or list comprehension can easily become inefficient and slow. If you only need the tokenization and lexical attributes, you can run nlp.make_doc instead, which will only run the tokenizer. For an additional speed boost, you can also use the nlp.tokenizer.pipe method, which will process the texts as a stream.

In [ ]:
from spacy.matcher import PhraseMatcher

matcher = PhraseMatcher(nlp.vocab, attr="LOWER")
# Only run nlp.make_doc to speed things up

icd_patterns = [nlp.make_doc(text) for text in ICD]
neg_patterns = [nlp.make_doc(text) for text in NEG]

matcher.add("SYM", icd_patterns)
matcher.add("NEG", neg_patterns)

In [ ]:
matches = matcher(doc)
entities = []
for match_id, start, end in matches:
    span = doc[start:end]
    class_id = nlp.vocab.strings[match_id]
    print(span.text, span.start_char, span.end_char, class_id)
    ent = {"start": span.start_char, "end": span.end_char, "label": class_id}
    entities.append(ent)
print(entities)

Changed in v3.0
As of spaCy v3.0, PhraseMatcher.add takes a list of patterns as the second argument (instead of a variable number of arguments). The on_match callback becomes an optional keyword argument.

```
patterns = [nlp("health care reform"), nlp("healthcare reform")]
- matcher.add("HEALTH", on_match, *patterns)
+ matcher.add("HEALTH", patterns, on_match=on_match)
``` 

## Add pipe entity ruler


### ENtity ruler
The entity ruler is designed to integrate with spaCy’s existing pipeline components and enhance the named entity recognizer. If it’s added before the "ner" component, the entity recognizer will respect the existing entity spans and adjust its predictions around it. This can significantly improve accuracy in some cases. If it’s added after the "ner" component, the entity ruler will only add spans to the doc.ents if they don’t overlap with existing entities predicted by the model. To overwrite overlapping entities, you can set overwrite_ents=True on initialization.

In [41]:
# Entity ruler
config = {
      "phrase_matcher_attr": "lower",
      "validate": True, 	# Whether patterns should be validated (passed to the Matcher and PhraseMatcher). Defaults to False.
      "overwrite_ents": False,
      "ent_id_sep": "||",
    }

try:
  ruler = nlp.add_pipe("entity_ruler", config=config)
  # ruler.phrase_matcher = matcher # Changes the phrsematcher
except (NameError, ValueError) as e:
  ruler = nlp.replace_pipe(name="entity_ruler", factory_name="entity_ruler", config=config)
  #ruler.phrase_matcher = matcher # Changes the phrsematcher
  print("Entity ruler already added, replacing instead of creating")

print("----Printing patterns, should be empty at this point----")    
print(ruler.patterns)

Entity ruler already added, replacing instead of creating
----Printing patterns, should be empty at this point----
[]


In [50]:
file =  data_path / 'raw/corpus/chart2.txt'
pre_doc = open(file, 'r').read()

doc = nlp(pre_doc)

print([(ent.text, ent.label_, ent.ent_id_) for ent in doc.ents])

[('dyspné', 'SYM', ''), ('Tidigare', 'TME', ''), ('Inga', 'NEG', ''), ('utan', 'NEG', ''), ('igår kväll', 'TME', ''), ('Ingen', 'NEG', ''), ('87%', 'MSR', ''), ('utan', 'NEG', ''), ('hemtjänsten', 'ORG', ''), ('3-4 dagar', 'TME', ''), ('94%', 'MSR', ''), ('inga', 'NEG', ''), ('inga', 'NEG', ''), ('Ingen', 'NEG', ''), ('ingen', 'NEG', ''), ('acidos', 'SYM', ''), ('utan', 'NEG', ''), ('Inga', 'NEG', ''), ('om 1 timme', 'TME', ''), ('ej', 'NEG', '')]


In [44]:
doc = nlp("Kolera, är inte en hemsk sjukdom")

print([(ent.text, ent.label_, ent.ent_id_) for ent in doc.ents])

[('Kolera', 'SYM', ''), ('inte', 'NEG', '')]


In [23]:
icd_entries = [nlp.make_doc(text.lower()) for text in ICD]
neg_phrases = [nlp.make_doc(text.lower()) for text in NEG]

In [42]:
patterns = []

for pattern in icd_entries:
  #if str(pattern) != "":
  p = {"label":"SYM","pattern":[{"LOWER":str(pattern)}]}
  # print(p)
  patterns.append(p)

for pattern in neg_phrases:
  #if str(pattern) != "":
  p = {"label":"NEG","pattern":[{"LOWER":str(pattern)}]}
  # print(p)
  patterns.append(p)

with nlp.select_pipes(enable="tagger"):
  ruler.add_patterns(patterns)



In [ ]:
nlp.pipeline

In [26]:
ruler.patterns

[{'label': 'SYM',
  'pattern': [{'LOWER': 'annan matförgiftning orsakad av bakterier som ej klassificeras på annan plats'}]},
 {'label': 'SYM', 'pattern': [{'LOWER': 'amöbainfektion'}]},
 {'label': 'SYM', 'pattern': [{'LOWER': 'andra protozosjukdomar i tarmen'}]},
 {'label': 'SYM',
  'pattern': [{'LOWER': 'tarminfektioner orsakade av virus och andra specificerade organismer'}]},
 {'label': 'SYM',
  'pattern': [{'LOWER': 'annan gastroenterit och kolit av infektiös och ospecificerad orsak'}]},
 {'label': 'SYM',
  'pattern': [{'LOWER': 'tuberkulos i andningsorganen, bakteriologiskt och histologiskt verifierad'}]},
 {'label': 'SYM',
  'pattern': [{'LOWER': 'tuberkulos i andningsorganen, ej verifierad bakteriologiskt eller histologiskt'}]},
 {'label': 'SYM', 'pattern': [{'LOWER': 'tuberkulos i nervsystemet'}]},
 {'label': 'SYM', 'pattern': [{'LOWER': 'tuberkulos i andra organ'}]},
 {'label': 'SYM',
  'pattern': [{'LOWER': 'miliartuberkulos (utspridd tuberkulos)'}]},
 {'label': 'SYM', 'patte

In [27]:
ruler.to_disk(Path(repo_path / "models/patterns.jsonl"))

In [28]:
# Analyse the pipeline and store the analysis under 'pipe_analysis'
pipe_analysis = nlp.analyze_pipes(pretty=True)


============================= Pipeline Overview =============================

#   Component         Assigns               Requires   Scores             Retokenizes
-   ---------------   -------------------   --------   ----------------   -----------
0   transformer       doc._.trf_data                                      False      
                                                                                     
1   tagger            token.tag                        tag_acc            False      
                                                                                     
2   parser            token.dep                        dep_uas            False      
                      token.head                       dep_las                       
                      token.is_sent_start              dep_las_per_type              
                      doc.sents                        sents_p                       
                                                       sents

In [ ]:
@Language.component("entity_ruler")
def custom_sentencizer(doc):
    for i, token in enumerate(doc[:-2]):
        # Define sentence start if pipe + titlecase token
        if token.text == "|" and doc[i + 1].is_title:
            doc[i + 1].is_sent_start = True
        else:
            # Explicitly set sentence start to False otherwise, to tell
            # the parser to leave those tokens alone
            doc[i + 1].is_sent_start = False
    return doc

In [ ]:
 def to_disk(
        self, path: Union[str, Path], *, exclude: Iterable[str] = SimpleFrozenList()
    ) -> None:
        """Save the entity ruler patterns to a directory. The patterns will be
        saved as newline-delimited JSON (JSONL).
        path (str / Path): The JSONL file to save.
        DOCS: https://spacy.io/api/entityruler#to_disk
        """
        path = ensure_path(path)
        cfg = {
            "overwrite": self.overwrite,
            "phrase_matcher_attr": self.phrase_matcher_attr,
            "ent_id_sep": self.ent_id_sep,
        }
        serializers = {
            "patterns": lambda p: srsly.write_jsonl(
                p.with_suffix(".jsonl"), self.patterns
            ),
            "cfg": lambda p: srsly.write_json(p, cfg),
        }
        if path.suffix == ".jsonl":  # user wants to save only JSONL
            srsly.write_jsonl(path, self.patterns)
        else:
            to_disk(path, serializers, {})

In [ ]:
print(nlp.pipe_names)  # see all components in the pipeline

## DISPLAY ENTITIES

In [51]:
from spacy import displacy

colors = {
          "SYM": "linear-gradient(90deg, #99154e, #99154e)",
          "NEG": "linear-gradient(90deg, #ffc93c, #ffc93c)"
          }
 

options = {"compact": False, 
           "bg": "#09a3d5",
           "color": "white",
           "font": "Source Sans Pro",
            "ents": ["SYM", "NEG", "TMN"], 
           "colors": colors}

displacy.render(doc, style="ent", jupyter=True, options=options)

In [53]:
from spacy import displacy

sentence_spans = list(doc.sents)

sym_sentences = []
neg_sentences = []

for sent in sentence_spans:
  ents = list(sent.ents)
  for ent in ents:
    if ent.label_ == "SYM":
      # print(ent.label_)
      sym_sentences.append(sent) 
      
    if ent.label_ == "NEG":
      # print(ent.label_)
      neg_sentences.append(sent) 

In [55]:
what_to_render = sym_sentences

options = {"compact": True, 
           "bg": "#09a3d5",
           "color": "white",
           "font": "Source Sans Pro",
           "fine_grained": True}

style = "dep"


displacy.render(what_to_render, style=style, jupyter=True, options=options)

In [ ]:
from tabulate import tabulate
import pandas as pd

cols = ("space", "text", "lemma", "normalization", "POS", "explain", "stopword", "dep","NE", "sentiment")
rows = []

for t in doc: 
  if t.ent_type_ != "":

    row = [t.is_space, t.text, t.lemma_, t.norm_, t.pos_, spacy.explain(t.pos_), t.is_stop, t.dep_, t.ent_type_, t.sentiment]
    rows.append(row)

df = pd.DataFrame(rows, columns=cols)
print(tabulate(df, headers = cols, tablefmt='psql', showindex=False))


+---------+-------------+-----------+-----------------+-------+------------+------------+-----------+------+-------------+
| space   | text        | lemma     | normalization   | POS   | explain    | stopword   | dep       | NE   |   sentiment |
|---------+-------------+-----------+-----------------+-------+------------+------------+-----------+------+-------------|
| False   | dyspné      | dyspné    | dyspné          | NOUN  | noun       | False      | conj      | SYM  |           0 |
| False   | Tidigare    | Tidigare  | tidigare        | ADJ   | adjective  | True       | amod      | TME  |           0 |
| False   | Inga        | Inga      | inga            | DET   | determiner | True       | det       | NEG  |           0 |
| False   | utan        | utan      | utan            | ADP   | adposition | True       | case      | NEG  |           0 |
| False   | igår        | igår      | igår            | ADV   | adverb     | True       | case      | TME  |           0 |
| False   | kväl

In [ ]:
import pandas as pd

cols = (
    "NE",
    "IOB", 
    "text", 
    "lemma", 
    "POS", 
    "dep",
    "right edge"
     )
df = pd.DataFrame()
rows = []

for t in doc:
      if t.ent_type_ == "SYM" or "NEG":
        row = [
               t.ent_type_,
               t.ent_iob_, 
               t.text, 
               t.lemma_, 
               spacy.explain(t.pos_), 
               t.dep_,
               t.right_edge
        ] 
              
        rows.append(row)

df = pd.DataFrame(rows, columns=cols)
print(tabulate(df, headers = cols, tablefmt='psql', showindex=False))      

+------+-------+---------------------+---------------------+---------------------------+------------+---------------------+
| NE   | IOB   | text                | lemma               | POS                       | dep        | right edge          |
|------+-------+---------------------+---------------------+---------------------------+------------+---------------------|
|      | O     | Kontaktorsak        | Kontaktorsak        | noun                      | ROOT       | :                   |
|      | O     | :                   | :                   | punctuation               | punct      | :                   |
|      | O     | Medicinlarm         | Medicinlarm         | noun                      | ROOT       |                     |
|      | O     | ,                   | ,                   | punctuation               | punct      | ,                   |
| SYM  | B     | dyspné              | dyspné              | noun                      | conj       | dyspné              |
|      |

* raw text
* lemma – a root form of the word
* part of speech
* a flag for whether the word is a stopword – i.e., a common word that may be filtered out

### inspect sentences and chunks

In [ ]:
for sent in doc.sents:
    print(">", sent)

> Kontaktorsak:
> Medicinlarm, dyspné

> Tidigare sjukdomar:
> Waranbehandlat FF, hyperlipidemi.
> BC vä 99, opererad mastektomi + lymfkörtelutrymning höger.
> Inga recidiv.
> NSTEMI 2009.
> Stroke 2012 utan större sequele.
> Minnesproblematik.
> 

> Aktuellt:
> Funnen i hemmet på golvet, oklart hur länge hon blivit liggande.
> Hittad av hemtjänst kl 7, senast tillsyn igår kväll ca kl 20.
> Ingen autoanamnes.
> I ambulansen febril, desaturerad 87% utan syrgas.
> Enl hemtjänsten varit förkyld 3-4 dagar.
> Stabila VP i övrigt.
> 

> A:
> Fritt
B:
> Antingsljud bilateralt, sat 94% på 5l syrgas.
> Obstruktiv, förlängt expirum.
> Erhåller inhalation sapimol.
> AF. 28.
> Stabil bröstkorg, inga tecken till trauma.

> C: Fina radialispulsar bilat, kall perifert.
> Marmorerad över knän.
> BT 90/70, p 119.
> Oregelbunden.
> I telemetri smala komplex.
> Erhåller infart, påbörjar vätskerecusitering.
> Stabilt bäcken.
> Buk mjuk och oöm.

> D:
> Liksidiga pupiller, inga fokala neurotiska bortfall.


In [ ]:
for chunk in doc.noun_chunks:
    print(chunk.text)

Kontaktorsak:
Medicinlarm, dyspné

Tidigare sjukdomar:
Waranbehandlat FF, hyperlipidemi.
BC vä 99, opererad mastektomi + lymfkörtelutrymning höger.
Inga recidiv.
NSTEMI 2009.
Stroke 2012 utan större sequele.
Minnesproblematik.
Aktuellt:
hon
kl
igår kväll
kl
Ingen autoanamnes.
hemtjänsten
Stabila VP i övrigt.
A:
Fritt
B:
Antingsljud bilateralt, sat 94% på 5l syrgas.
Obstruktiv, förlängt expirum.
inhalation
AF. 28.
Stabil bröstkorg, inga tecken till trauma.

C: Fina radialispulsar bilat, kall perifert.
BT 90/70, p 119.
I telemetri smala komplex.
infart
vätskerecusitering
Stabilt bäcken.
Buk
D:
Liksidiga pupiller, inga fokala neurotiska bortfall.
Ingen nackstelhet.
E:
Skrapsår höger kind, blåmärke vänster höft, ingen utåtrotation.
Hel hudskstym.
Temp
Lab:
Blodgas med akut respriatorisk acidos utan kompensation, laktat 8.
EKG:
Förmaksflimmer kammarfrekvens 123.
Bedömning:
Sepsis med oklart fokus samt fall med oklar traumamekanism.
Inga tecken till blödning i buken eller andra svåra traumat

## Acronyms

In [ ]:
import scispacy
from scispacy.abbreviation import AbbreviationDetector

# Add the abbreviation pipe to the spacy pipeline.
nlp.add_pipe("abbreviation_detector")

doc = nlp("""Förmaksflimmer (FF) \
          FF är den vanligaste takyarytmin. Förekomsten ökar med åldern, och är ca 1 % vid 50 år och > 10 % vid 80 år. Kan förekomma hos hjärtfriska individer, s k lone atrial fibrillation, men vanligen föreligger organisk hjärtsjukdom. Det är rekommenderat att dessa patienter handläggs på ett strukturerat sätt och med patienten som en viktig del i sitt team, ett personcentrerat arbetssätt, för bästa resultat.""")

doc = nlp("""Ett esofagus-EKG kan vara indicerat för att differentiera mellan FFl och andra regelbundna supraventrikulära takykardier, såsom AV nodal reentry-takykardi (AVNRT), WPW-syndrom eller ektopisk förmakstakykardi (EAT).""")
print("Abbreviation", "\t", "Definition")
for abrv in doc._.abbreviations:
	print(f"{abrv} \t ({abrv.start}, {abrv.end}) {abrv._.long_form}")

ValueError: ignored

In [ ]:
def replace_acronyms(text):
    doc = nlp(text)
    altered_tok = [tok.text for tok in doc]
    for abrv in doc._.abbreviations:
        altered_tok[abrv.start] = str(abrv._.long_form)

    return(" ".join(altered_tok))

In [ ]:
text = ("""Ett esofagus-EKG kan vara indicerat för att differentiera mellan FFl och andra regelbundna supraventrikulära takykardier, såsom AV nodal reentry-takykardi (AVNRT), WPW-syndrom eller ektopisk förmakstakykardi (EAT).""")

doc = nlp(text)

print("Abbreviation", "\t", "Definition")
for abrv in doc._.abbreviations:
	print(f"{abrv} \t ({abrv.start}, {abrv.end}) {abrv._.long_form}")
 
text = replace_acronyms(text)
doc = nlp(text)
print(text)

Abbreviation 	 Definition
AVNRT 	 (21, 22) AV nodal reentry-takykardi
EAT 	 (29, 30) ektopisk förmakstakykardi
Ett esofagus-EKG kan vara indicerat för att differentiera mellan FFl och andra regelbundna supraventrikulära takykardier , såsom AV nodal reentry-takykardi ( AV nodal reentry-takykardi ) , WPW-syndrom eller ektopisk förmakstakykardi ( ektopisk förmakstakykardi ) .


# ANALYSIS eXPERIMENTS

In [ ]:
# https://support.prodi.gy/t/evaluation-of-rule-based-matching/1431
true_positives = guesses.intersection(truth)
false_positives = guesses - truth
false_negatives = truth - guesses

precision = len(true_positives) / len(guesses)
recall = len(true_positives) / len(truth)
fscore = 2 * ((p * r) / (p + r + 1e-100))

When you make your sets, make sure that you’re representing the spans by the start and end offsets with the label, instead of just the text. It’s not so relevant in your case, but it covers you if you do have inputs with multiple annotations that have the same text content. A tuple (start, end, label) will be hashable, so you can store it in a set.

If you’re making the set over a whole dataset, you’ll also want to add in the input hash, to make sure you’re referring to the right examples. All up, it should be as easy as this:

In [ ]:
def get_annotations(dataset):
    annotations = set()
    for eg in dataset:
        for span in eg["spans"]:
            annotations.add((span["start"], span["end"], span["label"]))
    return annotations

# DB = connect()
truth = get_annotations(DB.get_dataset(gold_annotations))
guesses =  get_annotations(matcher_output)
scores = spacy.scorer.PRFScore()
scores.score_set(guesses, truth)
print(scores.precision, scores.recall, scores.fscore)

NameError: ignored

In [ ]:
# load an example dataset
from vega_datasets import data
cars = data.cars()

import altair as alt

points = alt.Chart(cars).mark_point().encode(
  x='Year:T',
  y='Miles_per_Gallon',
  color='Origin'
).properties(
  width=800
)

lines = alt.Chart(cars).mark_line().encode(
  x='Year:T',
  y='mean(Miles_per_Gallon)',
  color='Origin'
).properties(
  width=800
).interactive(bind_y=False)
              
points + lines

In [ ]:
print("Create a new set:")
x = set()
print(x)
print(type(x))
print("\nCreate a non empty set:")
n = set([0, 1, 2, 3, 4])
print(n)
print(type(n))
print("\nUsing a literal:")
a = {1,2,3,'foo','bar'}
print(type(a))
print(a)

Create a new set:
set()
<class 'set'>

Create a non empty set:
{0, 1, 2, 3, 4}
<class 'set'>

Using a literal:
<class 'set'>
{1, 2, 3, 'foo', 'bar'}


A set is an unordered collection of items. Every set element is unique (no duplicates) and must be immutable (cannot be changed).

However, a set itself is mutable. We can add or remove items from it.

Sets can also be used to perform mathematical set operations like union, intersection, symmetric difference, etc.

In [ ]:
# Training data for a part-of-speech tagger
doc = Doc(vocab, words=["I", "like", "stuff"])
gold_dict = {"tags": ["NOUN", "VERB", "NOUN"]}
example = Example.from_dict(doc, gold_dict)

# Training data for an entity recognizer (option 1)
doc = nlp("Laura flew to Silicon Valley.")
gold_dict = {"entities": ["U-PERS", "O", "O", "B-LOC", "L-LOC"]}
example = Example.from_dict(doc, gold_dict)

# Training data for an entity recognizer (option 2)
doc = nlp("Laura flew to Silicon Valley.")
gold_dict = {"entities": [(0, 5, "PERSON"), (14, 28, "LOC")]}
example = Example.from_dict(doc, gold_dict)

# Training data for text categorization
doc = nlp("I'm pretty happy about that!")
gold_dict = {"cats": {"POSITIVE": 1.0, "NEGATIVE": 0.0}}
example = Example.from_dict(doc, gold_dict)

# Training data for an Entity Linking component (also requires entities & sentences)
doc = nlp("Russ Cochran his reprints include EC Comics.")
gold_dict = {"entities": [(0, 12, "PERSON")],
             "links": {(0, 12): {"Q7381115": 1.0, "Q2146908": 0.0}},
             "sent_starts": [1, -1, -1, -1, -1, -1, -1, -1]}
example = Example.from_dict(doc, gold_dict)

NameError: ignored

In [ ]:
%cd clinical_NLP_SE/
!git pull

/content/clinical_NLP_SE
remote: Enumerating objects: 30, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 26 (delta 3), reused 26 (delta 3), pack-reused 0
Unpacking objects: 100% (26/26), done.
From https://github.com/callebalik/clinical_NLP_SE
   212afca..76e8d07  master     -> origin/master
Updating 212afca..76e8d07
Fast-forward
 data/raw/corpus/conll2003/chart1.txt/admin.conll | 321 ++++++++++++++++
 data/raw/corpus/conll2003/chart2.txt/admin.conll | 366 ++++++++++++++++++
 data/raw/corpus/conll2003/chart3.txt/admin.conll | 280 ++++++++++++++
 data/raw/corpus/conll2003/chart4.txt/admin.conll | 451 +++++++++++++++++++++++
 data/raw/corpus/conll2003/chart5.txt/admin.conll | 214 +++++++++++
 data/raw/corpus/conll2003/pat1.txt/admin.conll   | 220 +++++++++++
 data/raw/corpus/conll2003/pat2.txt/admin.conll   | 173 +++++++++
 data/raw/corpus/conll2003/pat3.txt/admin.conll   | 173 +++++++++
 data/raw/corpus/conll2003/pat4.t

In [33]:
!python -m spacy convert --converter ner /content/clinical_NLP_SE/data/raw/corpus/conll2003/chart1.txt/admin.conll /content/clinical_NLP_SE/data/interim

2021-05-31 07:20:37.358995: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
ℹ Auto-detected token-per-line NER format
ℹ Grouping every 1 sentences into a document.
⚠ To generate better training data, you may want to group sentences
into documents with `-n 10`.
✔ Generated output file (34 documents):
/content/clinical_NLP_SE/data/interim/admin.spacy


In [ ]:
python -m spacy debug-data de /content/clinical_NLP_SE/data/interim/admin.spacy -p ner -b de_core_news_md

SyntaxError: ignored

In [ ]:
!pip install conllu

In [ ]:
 from conllu import parse
 from io import open
from conllu import parse_incr

data_file = open("/content/clinical_NLP_SE/data/raw/corpus/mockup-patient-records/chart1.txt/admin.conllu", "r", encoding="utf-8")
for tokenlist in parse_incr(data_file):
    print(tokenlist)

TokenList<Kontaktorsak, :, Dyspné, Tidigare, sjukdomar, :, Insulin, och, tablettbehandlad, DM2, .>
TokenList<Tablettbehandlad, hypertoni, .>
TokenList<Höga, blodfetter, .>
TokenList<Socialt, :, Ensamboende, ,, hemtjänst, *, 1, .>
TokenList<Aktuellt, :, Insjuknat, för, tre, dagar, sedan, med, andnöd, vid, gång, .>
TokenList<Tillkomst, av, hosta, under, gårdagen, .>
TokenList<Ej, noterat, feber, ,, men, ej, eller, mätt, tempen, .>
TokenList<Idag, nedsatt, AT, ,, svårt, att, gå, .>
TokenList<Tagit, sina, mediciner, som, vanligt, .>
TokenList<Sökt, VC, som, hänvisat, hit, pga, destruering, .>
TokenList<Inte, haft, några, utslag, .>
TokenList<Ej, kontakt, med, någon, annan, som, varit, sjuk, .>
TokenList<Ej, varit, ute, och, rest, .>
TokenList<Ingen, smärta, vid, miktion, ,, buksmärta, eller, ÖNH, symptom, .>
TokenList<Idag, sämre, ,, kan, ej, mobilisera, sig, i, hemmet, .>
TokenList<Inkommer, på, remiss, .>
TokenList<Inga, kräkningar, eller, GI, symptom, .>
TokenList<Status, :, AT, :, Gott

In [ ]:
from spacy.tokens import DocBin
from spacy.training import Corpus

doc_bin = DocBin(docs=docs)
doc_bin.to_disk("./data.spacy")
reader = Corpus("./data.spacy")

AttributeError: ignored

In [ ]:

doc = nlp(/content/clinical_NLP_SE/data/interim/admin.spacy)

SyntaxError: ignored

In [ ]:
!python -m spacy debug-data --help

2021-05-28 10:51:12.555487: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
Usage: python -m spacy debug-data 
           [OPTIONS] CONFIG_PATH

  Analyze, debug and validate your training and
  development data. Outputs useful stats, and
  can help you find problems like invalid entity
  annotations, cyclic dependencies, low data
  labels and more.

  DOCS: https://spacy.io/api/cli#debug-data

Arguments:
  CONFIG_PATH  Path to config file  [required]

Options:
  -c, --code-path, --code PATH  Path to Python
                                file with
                                additional code
                                (registered
                                functions) to be
                                imported

  -IW, --ignore-warnings        Ignore warnings,
                                only show stats
                                and errors
                                [default: False]

  

In [ ]:
import pandas as pd

cols = ("space", "text", "lemma", "normalization", "POS", "explain", "stopword", "dep","NE", "sentiment")
rows = []

for t in doc:
      if not t.is_space:
        row = [t.is_space, t.text, t.lemma_, t.norm_, t.pos_, spacy.explain(t.pos_), t.is_stop, t.dep_, t.ent_type_, t.sentiment]
        rows.append(row)

df = pd.DataFrame(rows, columns=cols)
    
df


,space,text,lemma,normalization,POS,explain,stopword,dep,NE,sentiment
0,False,data_path,data_path,data_path,NOUN,noun,False,ROOT,,0.0
1,False,/,/,/,SYM,symbol,False,cc,,0.0
2,False,interim,interim,interim,NOUN,noun,False,conj,,0.0
3,False,/,/,/,SYM,symbol,False,cc,,0.0
4,False,admin.spacy,admin.spacy,admin.spacy,NOUN,noun,False,conj,,0.0


In [ ]:
!python -m spacy convert /content/clinical_NLP_SE/data/raw/corpus/mockup-patient-records/chart1.txt/admin.conllu /path/to/output/doc.jsonl -c conllu.

SyntaxError: ignored


# ANALYSIS

In [34]:
%cd clinical_NLP_SE/data/
!python -m spacy convert raw/conll2003 processed/ --converter conll --n-sents 100

/content/clinical_NLP_SE/data
2021-05-31 07:21:02.046990: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
ℹ Grouping every 100 sentences into a document.
✔ Generated output file (1 documents): processed/chart1.spacy
ℹ Grouping every 100 sentences into a document.
✔ Generated output file (1 documents): processed/chart2.spacy
ℹ Grouping every 100 sentences into a document.
✔ Generated output file (1 documents): processed/chart3.spacy
ℹ Grouping every 100 sentences into a document.
✔ Generated output file (1 documents): processed/chart4.spacy
ℹ Grouping every 100 sentences into a document.
✔ Generated output file (1 documents): processed/chart5.spacy
ℹ Grouping every 100 sentences into a document.
✔ Generated output file (1 documents): processed/pat1.spacy
ℹ Grouping every 100 sentences into a document.
✔ Generated output file (1 documents): processed/pat2.spacy
ℹ Grouping every 100 sentences into a document.
✔ Generate

In [56]:
#!rm -r /content/clinical_NLP_SE/models/ 
#!mkdir /content/clinical_NLP_SE/models/
%cd /content/clinical_NLP_SE/models/
nlp.to_disk(repo_path / 'models/test_3/')
!python -m spacy evaluate /content/clinical_NLP_SE/models/test_3/ ../data/processed/ --output models/metrics.json --displacy-path ../models/ --gpu-id 0

/content/clinical_NLP_SE/models
2021-05-31 07:44:32.297238: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
ℹ Using GPU: 0

================================== Results ==================================

TOK      -    
TAG      0.00 
POS      -    
LEMMA    -    
UAS      -    
LAS      -    
NER P    7.69 
NER R    26.19
NER F    11.89
SENT P   28.05
SENT R   51.05
SENT F   36.20
SPEED    668  


=============================== NER (per type) ===============================

                 P        R       F
SYM          13.33     7.41    9.52
TME           0.00     0.00    0.00
NEG          11.69   100.00   20.93
DIS           0.00     0.00    0.00
ORG           0.00     0.00    0.00
null          0.00     0.00    0.00
PRS           0.00     0.00    0.00
LOC           0.00     0.00    0.00
NEGATIONER    0.00     0.00    0.00
MSR           0.00     0.00    0.00

✔ Generated 25 parses as HTML
../models
Traceback (mo

In [57]:
import IPython
# IPython.display.HTML(filename=Path(repo_path / 'models/parses.html'))
IPython.display.HTML(filename=Path(repo_path / 'models/entities.html'))

In [ ]:
from spacy.scorer import Scorer
from spacy.tokens import Doc
from spacy.training import Example

# Training data for an entity recognizer (option 2)
doc = nlp("Laura flew to Silicon Valley.")
gold_dict = {"entities": [(0, 5, "PRS"), (14, 28, "LOC")]}
example = Example.from_dict(doc, gold_dict)

print(example.text)
examples = []
examples.append(example)

# Spacy V3 The Language.evaluate method now takes a batch of Example objects instead of tuples of Doc and GoldParse objects.
scores = nlp.evaluate(examples)
print(scores["ents_p"])


# Default scoring pipeline
#scorer = Scorer()

# Provided scoring pipeline
scores = Scorer.score_tokenization(examples)
print(scores)


# Returns A dictionary containing the PRF scores under the keys {attr}_p, {attr}_r, {attr}_f and the per-type PRF scores under {attr}_per_type
spans = Scorer.score_spans(examples, "ents")
print(spans["ents_per_type"])

from spacy import displacy

colors = {
          "SYM": "linear-gradient(90deg, #99154e, #99154e)",
          "NEG": "linear-gradient(90deg, #ffc93c, #ffc93c)"
          }
 

options = {"compact": False, 
           "bg": "#09a3d5",
           "color": "white",
           "font": "Source Sans Pro"}

displacy.render(doc, style="ent", jupyter=True, options=options)

print(example.to_dict())

Laura flew to Silicon Valley.
1.0
{'token_acc': 1.0, 'token_p': 1.0, 'token_r': 1.0, 'token_f': 1.0}
{'PRS': {'p': 1.0, 'r': 1.0, 'f': 1.0}, 'LOC': {'p': 1.0, 'r': 1.0, 'f': 1.0}}


{'doc_annotation': {'cats': {}, 'entities': ['U-PRS', 'O', 'O', 'B-LOC', 'L-LOC', 'O'], 'links': {}}, 'token_annotation': {'ORTH': ['Laura', 'flew', 'to', 'Silicon', 'Valley', '.'], 'SPACY': [True, True, True, True, False, False], 'TAG': ['', '', '', '', '', ''], 'LEMMA': ['', '', '', '', '', ''], 'POS': ['', '', '', '', '', ''], 'MORPH': ['', '', '', '', '', ''], 'HEAD': [0, 1, 2, 3, 4, 5], 'DEP': ['', '', '', '', '', ''], 'SENT_START': [1, 0, 0, 0, 0, 0]}}


The current implementation of the alignment algorithm assumes that both tokenizations add up to the same string. For example, you’ll be able to align ["I", "'", "m"] and ["I", "'m"], which both add up to "I'm", but not ["I", "'m"] and ["I", "am"].

In [ ]:
from spacy.training import Alignment

bert_tokens = ["obama", "'", "s", "podcast"]
spacy_tokens = ["obama", "'s", "podcast"]
alignment = Alignment.from_strings(bert_tokens, spacy_tokens)
a2b = alignment.x2y
assert list(a2b.dataXd) == [0, 1, 1, 2]
print(list(a2b.dataXd) == [0, 1, 1, 2])

# If a2b.dataXd[1] == a2b.dataXd[2] == 1, that means that A[1] ("'") and A[2] ("s") both align to B[1] ("'s")



True
